In [1]:
import pandas as pd
import itertools
import numpy as np
from scipy.stats import qmc
import matplotlib.pyplot as plt
import csv
import time
import sys
from phasepy import mixture, component, virialgamma, preos
from phasepy.equilibrium import tpd_min, tpd_minimas, lle_init, sle, slle

In [2]:
water = component(name='water', Tc=647.096, Pc=220.64, Zc=0.229, Vc=55.9472,
                  GC={'H2O':1}) # water

npentane = component(name='n-pentane', Tc=469.7, Pc=33.7, Zc=0.27, Vc=313,
                     GC={'CH3':2, 'CH2':3}) # n-pentane

nheptane = component(name='n-heptane', Tc=540.2, Pc=27.4, Zc=0.261, Vc=428,
                     GC={'CH3':2, 'CH2':5}) # n-heptane

ethanol = component(name='ethanol', Tc=514, Pc=61.37, Zc=0.241, Vc=168,
                    GC={'CH3':1, 'CH2':1, 'OH':1}) # ethanol

_1propanol = component(name='1-propanol', Tc=536.8, Pc=51.69, Zc=0.254, Vc=219,
                       GC={'CH3':1, 'CH2':2, 'OH':1}) # 1-propanol

_1butanol = component(name='1-butanol', Tc=563.1, Pc=44.14, Zc=0.258, Vc=273,
                      GC={'CH3':1, 'CH2':3, 'OH':1})

_1pentanol = component(name='1-pentanol', Tc=588.1, Pc=38.97, Zc=0.258, Vc=326,
                       GC={'CH3':1, 'CH2':4, 'OH':1})

acetone = component(name='acetone', Tc=508.1, Pc=47, Zc=0.237, Vc=213,
                    GC={'CH3CO':1, 'CH3':1})
ibuprofen = component(name='ibuprofen', Tc=765, Pc=29.8, Zc=0.313, Vc=668,
                      GC={'CH3':3, 'CH':1, 'ACH':4, 'ACCH2':1, 'ACCH':1, 'COOH':1},
                      dHf=25500, Tf=347.15)

In [3]:
component_map = {"ethanol" : ethanol, "1-propanol" : _1propanol, "acetone" : acetone, "1-butanol" : _1butanol, 
                 "1-pentanol" : _1pentanol, "n-pentane" : npentane, "n-heptane" : nheptane, "water" : water}

In [4]:
component_list = [water,npentane,nheptane,ethanol,_1propanol,_1butanol,_1pentanol,acetone]
binary_mix = itertools.permutations(component_list, 2)

In [5]:
def return_eos(c1,c2):
    mix_func = mixture(c1,c2)
    mix_func += ibuprofen
    mix_func.original_unifac()
    eos = virialgamma(mix_func, virialmodel='ideal_gas', actmodel='original_unifac')
    return eos

In [6]:
eos_dict = {}
counter=0
for i in binary_mix:
    eos = return_eos(i[0],i[1])
    eos_dict[counter] = eos
    counter+=1

In [7]:
def return_phases(b,t):
    liq_flag = 0
    liq_liq_flag = 0
    sol_liq_flag = 0
    sol_liq_liq_flag = 0
    if (b[0] == 1 and b[1] == 0 and b[2] == 0 and t[0] == 0 and t[1] >= 0) or (b[0] == 0 and b[1] == 1 and b[2] == 0 and t[0] == 0 and t[1] >= 0) or (b[0] == 1 and b[1] == 0 and b[2] == 0 and t[0] > 0 and t[1] > 0) or (b[0] == 0 and b[1] == 1 and b[2] == 0 and t[0] > 0 and t[1] > 0):
            liq_flag = 1
    elif (b[0] >= 0 and b[1] >= 0 and b[2] == 0 and t[0] == 0 and t[1] >= 0):
            liq_liq_flag = 1
    elif (b[0] > 0 and b[1]==0 and b[2]>0 and t[0]==0 and t[1]==0) or (b[0] == 0 and b[1]>0 and b[2]>0 and t[0]==0 and t[1]==0) or (b[0] > 0 and b[1]==0 and b[2]>0 and t[0]>0 and t[1]==0) or (b[0] == 0 and b[1]>0 and b[2]>0 and t[0]>0 and t[1]==0):
            sol_liq_flag = 1
    elif (b[0] > 0 and b[1] > 0 and b[2] > 0 and t[0] == 0 and t[1] == 0):
            sol_liq_liq_flag = 1
    return liq_flag, liq_liq_flag, sol_liq_flag, sol_liq_liq_flag

In [8]:
df_part = pd.read_csv('Case2.csv', nrows=4096)
z1 = df_part['x1']
z2 = df_part['x2']
z3 = 1 - z1 - z2
z = np.column_stack((z1, z2, z3))
T = df_part['T']

In [9]:
st = time.time() #start timer
nmin = 3
P = 1.01325
ismisc = []
for i in eos_dict:
    for j in range(4096):
        tpd_m = slle(z[j], T[j], P, eos_dict[i], solid_phases_index=[2], full_output=True, accelerate_every=20)
        theta = tpd_m.tetha
        betas = tpd_m.beta
        liq_flag, liq_liq_flag, sol_liq_flag, sol_liq_liq_flag = return_phases(betas,theta)
        if liq_flag == 1 or sol_liq_flag == 1:
            ismisc.append(1)
        elif liq_liq_flag == 1 or sol_liq_liq_flag == 1:
            ismisc.append(0)
        else:
            print(z[j],T[j])
            print(tpd_m)
            ismisc.append(2)

et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds') #execution time

c:\users\tk3016\phasepy\phasepy\equilibrium\multiflash.py:19: RuntimeWarning: divide by zero encountered in divide
  Xref = Z/sum1
C:\Users\tk3016\Anaconda3\envs\phasepy\lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
c:\users\tk3016\phasepy\phasepy\equilibrium\multiflash.py:36: RuntimeWarning: invalid value encountered in matmul
  dfb = - ter1@K1
c:\users\tk3016\phasepy\phasepy\equilibrium\multiflash.py:38: RuntimeWarning: invalid value encountered in multiply
  dft = Id * (Xref*Kexp.T).sum(axis=1)
c:\users\tk3016\phasepy\phasepy\equilibrium\multiflash.py:40: RuntimeWarning: invalid value encountered in matmul
  dft -= ter1 @ ter2
c:\users\tk3016\phasepy\phasepy\equilibrium\solidequilibria.py:172: RuntimeWarning: invalid value encountered in multiply
  X[1:(n_fluid)] = (Xref*Kexp.T)[:(n_fluid-1)]
c:\users\tk3016\phasepy\phasepy\equilibrium\solidequilibria.py:174: RuntimeWar

[0.04403583 0.04174612 0.91421805] 315.608776505664
            T: 315.608776505664
            P: 1.01325
  error_outer: nan
  error_inner: nan
         iter: 34
         beta: array([nan, nan, nan])
        tetha: array([nan, nan])
      X_fluid: array([[nan, nan,  0.],
       [nan, nan,  0.]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])
[0.03080176 0.04798243 0.92121581] 308.2663042988628
            T: 308.2663042988628
            P: 1.01325
  error_outer: 3.8407036651654354e-14
  error_inner: 9215908175089.027
         iter: 16
         beta: array([0., 0., 1.])
        tetha: array([8.53009217e-05, 0.00000000e+00])
      X_fluid: array([[3.90963693e-01, 6.09036307e-01, 3.85319819e-15],
       [3.90963673e-01, 6.09036327e-01, 3.85319935e-15]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])
[0.69241726 0.09096814 0.21661461] 306.0039480967447
            T: 306.0039480967447
            P: 1

c:\users\tk3016\phasepy\phasepy\equilibrium\multiflash.py:40: RuntimeWarning: invalid value encountered in subtract
  dft -= ter1 @ ter2


[0.00910702 0.0494863  0.94140668] 313.29540042728183
            T: 313.29540042728183
            P: 1.01325
  error_outer: nan
  error_inner: inf
         iter: 2
         beta: array([nan, nan, nan])
        tetha: array([nan, nan])
      X_fluid: array([[nan, nan,  0.],
       [nan, nan,  0.]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])
[0.0698412  0.00860702 0.92155178] 299.1933231526985
            T: 299.1933231526985
            P: 1.01325
  error_outer: 1.480789352473077e-12
  error_inner: 30581241548300.574
         iter: 37
         beta: array([0., 0., 1.])
        tetha: array([2.54467173e-05, 0.00000000e+00])
      X_fluid: array([[8.94372663e-01, 1.05627337e-01, 1.66302011e-18],
       [8.94372594e-01, 1.05627406e-01, 1.66302516e-18]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])


c:\users\tk3016\phasepy\phasepy\equilibrium\multiflash.py:16: RuntimeWarning: overflow encountered in exp
  Kexp = K.T*np.exp(tetha)
c:\users\tk3016\phasepy\phasepy\equilibrium\multiflash.py:16: RuntimeWarning: invalid value encountered in multiply
  Kexp = K.T*np.exp(tetha)
c:\users\tk3016\phasepy\phasepy\equilibrium\multiflash.py:22: RuntimeWarning: invalid value encountered in multiply
  f2 = (K1.T*Xref).sum(axis=1)
c:\users\tk3016\phasepy\phasepy\equilibrium\multiflash.py:35: RuntimeWarning: invalid value encountered in multiply
  ter1 = Xref/sum1 * K1.T


...

[0.05368046 0.03875597 0.90756357] 312.9796540416777
            T: 312.9796540416777
            P: 1.01325
  error_outer: nan
  error_inner: inf
         iter: 2
         beta: array([nan, nan, nan])
        tetha: array([nan, nan])
      X_fluid: array([[nan, nan,  0.],
       [nan, nan,  0.]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])
[0.00910702 0.0494863  0.94140668] 313.29540042728183
            T: 313.29540042728183
            P: 1.01325
  error_outer: nan
  error_inner: inf
         iter: 2
         beta: array([nan, nan, nan])
        tetha: array([nan, nan])
      X_fluid: array([[nan, nan,  0.],
       [nan, nan,  0.]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])
[0.04403583 0.04174612 0.91421805] 315.608776505664
            T: 315.608776505664
            P: 1.01325
  error_outer: nan
  error_inner: inf
         iter: 2
         beta: array([nan, nan, nan])
        tetha: arr

[0.01619275 0.00770624 0.97610101] 301.064977078326
            T: 301.064977078326
            P: 1.01325
  error_outer: nan
  error_inner: inf
         iter: 2
         beta: array([nan, nan, nan])
        tetha: array([nan, nan])
      X_fluid: array([[nan, nan,  0.],
       [nan, nan,  0.]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])
[0.01051484 0.07894973 0.91053543] 310.7518092781305
            T: 310.7518092781305
            P: 1.01325
  error_outer: nan
  error_inner: inf
         iter: 2
         beta: array([nan, nan, nan])
        tetha: array([nan, nan])
      X_fluid: array([[nan, nan,  0.],
       [nan, nan,  0.]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])
[0.05368046 0.03875597 0.90756357] 312.9796540416777
            T: 312.9796540416777
            P: 1.01325
  error_outer: nan
  error_inner: inf
         iter: 2
         beta: array([nan, nan, nan])
        tetha: array

[0.934371   0.06445302 0.00117598] 313.3633497223258
            T: 313.3633497223258
            P: 1.01325
  error_outer: nan
  error_inner: nan
         iter: 1
         beta: array([nan, nan, nan])
        tetha: array([nan, nan])
      X_fluid: array([[nan, nan, nan],
       [nan, nan, nan]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])
[0.03080176 0.04798243 0.92121581] 308.2663042988628
            T: 308.2663042988628
            P: 1.01325
  error_outer: nan
  error_inner: inf
         iter: 2
         beta: array([nan, nan, nan])
        tetha: array([nan, nan])
      X_fluid: array([[nan, nan,  0.],
       [nan, nan,  0.]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])
[0.00910702 0.0494863  0.94140668] 313.29540042728183
            T: 313.29540042728183
            P: 1.01325
  error_outer: nan
  error_inner: inf
         iter: 2
         beta: array([nan, nan, nan])
        tetha: a

[0.00910702 0.0494863  0.94140668] 313.29540042728183
            T: 313.29540042728183
            P: 1.01325
  error_outer: nan
  error_inner: inf
         iter: 2
         beta: array([nan, nan, nan])
        tetha: array([nan, nan])
      X_fluid: array([[nan, nan,  0.],
       [nan, nan,  0.]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])
[0.04403583 0.04174612 0.91421805] 315.608776505664
            T: 315.608776505664
            P: 1.01325
  error_outer: nan
  error_inner: inf
         iter: 2
         beta: array([nan, nan, nan])
        tetha: array([nan, nan])
      X_fluid: array([[nan, nan,  0.],
       [nan, nan,  0.]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])
[0.03716108 0.02245002 0.94038891] 305.6472483841702
            T: 305.6472483841702
            P: 1.01325
  error_outer: nan
  error_inner: inf
         iter: 2
         beta: array([nan, nan, nan])
        tetha: arr

[4.84091825e-01 5.15793724e-01 1.14450469e-04] 293.214675277099
            T: 293.214675277099
            P: 1.01325
  error_outer: nan
  error_inner: nan
         iter: 1
         beta: array([nan, nan, nan])
        tetha: array([nan, nan])
      X_fluid: array([[nan, nan, nan],
       [nan, nan, nan]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])
[0.00376001 0.09781981 0.89842019] 317.8792481129989
            T: 317.8792481129989
            P: 1.01325
  error_outer: nan
  error_inner: inf
         iter: 2
         beta: array([nan, nan, nan])
        tetha: array([nan, nan])
      X_fluid: array([[nan, nan,  0.],
       [nan, nan,  0.]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[0., 0., 1.]])
[0.01619275 0.00770624 0.97610101] 301.064977078326
            T: 301.064977078326
            P: 1.01325
  error_outer: nan
  error_inner: inf
         iter: 2
         beta: array([nan, nan, nan])
        te

In [10]:
print(len(ismisc))

229376


In [11]:
df = pd.read_csv('Case2.csv')
df['ismisc'] = ismisc
df2 = df[df.ismisc != 2]
df2

,Unnamed: 0,Solvent 1,Solvent 2,q1,q2,r1,r2,x1,x2,T,ismisc
0,0,water,n-pentane,1.400,3.316,0.920,3.825,0.079756,0.575261,303.718084,0
1,1,water,n-pentane,1.400,3.316,0.920,3.825,0.536915,0.078372,317.826495,0
2,2,water,n-pentane,1.400,3.316,0.920,3.825,0.369146,0.251464,296.909001,0
3,3,water,n-pentane,1.400,3.316,0.920,3.825,0.444403,0.388747,311.011016,0
4,4,water,n-pentane,1.400,3.316,0.920,3.825,0.243360,0.546032,308.531906,0
...,...,...,...,...,...,...,...,...,...,...,...
229371,229371,acetone,1-pentanol,2.336,4.208,2.574,4.599,0.893343,0.026755,303.505147,1
229372,229372,acetone,1-pentanol,2.336,4.208,2.574,4.599,0.275730,0.474450,299.539736,1
229373,229373,acetone,1-pentanol,2.336,4.208,2.574,4.599,0.497270,0.151272,313.654054,1
229374,229374,acetone,1-pentanol,2.336,4.208,2.574,4.599,0.413987,0.456378,294.238012,1


In [12]:
df2.to_csv('Case2_tpd_calc_slle.csv')